In [ ]:
from integration_test import web_url, test_username, do_api_call_v2, login, logout, set_slider_value, get_driver
import numpy as np
import requests
from time import sleep
from uuid import uuid1
import shortuuid
from backend.rating_fields import VIDEO_FIELDS, VIDEO_REPORT_FIELDS
from integration_test import web_url, test_username, login, logout, set_slider_value, do_api_call_v2, create_test_video

import os

# for adding the videos to DB
# don't use at the same time with the server running
# https://stackoverflow.com/questions/59119396/how-to-use-django-3-0-orm-in-a-jupyter-notebook-without-triggering-the-async-con
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from backend.models import Video, EmailDomain

import datetime
from contextlib import contextmanager


from unittest.mock import MagicMock
django_db_blocker = MagicMock()

In [ ]:
driver = get_driver()
login(driver)

In [ ]:
def _test_domains(driver, ack_email, rej_email, pend_email):
    to_check = [(ack_email, 'accepted_domains_class'),
                 (rej_email, 'rejected_domains_class'),
                 (pend_email, 'pending_domains_class')]

    for domain, elem_id in to_check:
        links = driver.find_element_by_class_name(elem_id).find_elements_by_class_name('domain_link_class')
        print(links)
        global link
        link = links[0]
        print(dir(links[0]), links[0].text, links[0].tag_name, ack_email)
        assert any([link.text == domain for link in links])

@contextmanager
def _create_emails(django_db_blocker):
    ack_email = f"@{uuid1()}.com"
    rej_email = f"@{uuid1()}.org"
    pend_email = f"@{uuid1()}.io"

    # creating email domains
    with django_db_blocker.unblock():
        EmailDomain.objects.create(domain=ack_email, status=EmailDomain.STATUS_ACCEPTED)
        EmailDomain.objects.create(domain=rej_email, status=EmailDomain.STATUS_REJECTED)
        EmailDomain.objects.create(domain=pend_email, status=EmailDomain.STATUS_PENDING)

    yield ack_email, rej_email, pend_email

    # deleting email domains
    with django_db_blocker.unblock():
        EmailDomain.objects.filter(domain__in=[ack_email, rej_email, pend_email]).delete()

In [ ]:
# going to user personal profile
driver.find_element_by_id('personal_info_menu').click()

sleep(2)

# going to the list of domains
driver.find_element_by_class_name('to_verified_domains_class').click()
sleep(1)


In [ ]:
ack_email

In [ ]:
driver.refresh()

In [ ]:


with _create_emails(django_db_blocker) as domains:
    ack_email, rej_email, pend_email = domains
    driver.refresh()
    _test_domains(driver, ack_email, rej_email, pend_email)



In [ ]:
link.text